In [42]:
import json
import os
import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
from tqdm import tqdm

In [43]:
knb_cache = '/share/lyc/knb-dict-n/'
# 1
# dataset='counterfact'
# dataset='zsre'
dataset='recent' 
# 2
model = "Qwen2-7B-Instruct"
knb_layer='this_layer'
# 3
# weight_name_list = ['attn.k_proj', 'attn.v_proj', 'attn.q_proj', 'attn.out_proj', 'mlp.fc_in', 'mlp.fc_out']
# weight_name_list = ['mlp.fc_in', 'mlp.fc_out']
# model = 'llama2-7b-chat'
# weight_name_list = ['self_attn.q_proj', 'self_attn.k_proj', 'self_attn.v_proj', 'self_attn.o_proj', 'mlp.up_proj', 'mlp.gate_proj', 'mlp.down_proj']
# weight_name_list = ['mlp.up_proj', 'mlp.gate_proj', 'mlp.down_proj']
weight_name_list = ['mlp.up_proj', 'mlp.down_proj']

In [44]:
knb_gas = {}
for weight_name in weight_name_list:
    root_dir = (
        f"/share/lyc/gas/{model}_{dataset}_answer_next_token_target_new/{weight_name}"
    )
    print(root_dir)
    # gas_weight_name, knb_weight_name = 'mlp.fc_in', 'mlp.fc_out'
    # gas_weight_name = knb_weight_name = weight_name
    file_name_list = os.listdir(root_dir)
    file_name_list.sort()
    print(file_name_list)
    # 拼接所有gas
    lines_all = []
    for file_name in tqdm(file_name_list):
        # print(file_name)
        path = os.path.join(root_dir, file_name)
        with open(path, 'r') as f:
            lines = f.readlines()
        lines_all.extend(lines)
    print(len(lines_all)) # 885 cf
    # 转换为numpy array格式
    gas_list = []
    for line in tqdm(lines_all):
        gas = json.loads(line)
        gas = np.array(gas)
        gas_list.append(gas)
    gas_array = np.array(gas_list)
    print(gas_array.shape) # (885, 28, 16384)
    knb_gas[weight_name] = gas_array

/share/lyc/gas/Qwen2-7B-Instruct_recent_answer_next_token_target_new/mlp.up_proj
['0-100-Qwen2-7B-Instruct-0,28.json', '100-200-Qwen2-7B-Instruct-0,28.json', '200-300-Qwen2-7B-Instruct-0,28.json', '300-400-Qwen2-7B-Instruct-0,28.json']


100%|██████████| 4/4 [00:24<00:00,  6.14s/it]


400


100%|██████████| 400/400 [01:35<00:00,  4.19it/s]


(400, 28, 18944)
/share/lyc/gas/Qwen2-7B-Instruct_recent_answer_next_token_target_new/mlp.down_proj
['0-100-Qwen2-7B-Instruct-0,28.json', '100-200-Qwen2-7B-Instruct-0,28.json', '200-300-Qwen2-7B-Instruct-0,28.json', '300-400-Qwen2-7B-Instruct-0,28.json']


100%|██████████| 4/4 [00:04<00:00,  1.16s/it]


400


100%|██████████| 400/400 [00:17<00:00, 23.39it/s]


(400, 28, 3584)


In [45]:
# 创建knb path
full_path = os.path.join(
    knb_cache, f"/home/yantao/llm2024/knb-dict-n/{model}/{dataset}/{knb_layer}"
)
if not os.path.exists(full_path):
    os.makedirs(full_path, exist_ok=True)
print(full_path)

/home/yantao/llm2024/knb-dict-n/Qwen2-7B-Instruct/recent/this_layer


In [46]:
# 知识定位的数据条数：100~all
n = gas_array.shape[0]
n_layers = gas_array.shape[1]
# 分位数,选取gas分数最高的1-p%作为knb
# p
p_list = [90,91,92,93,94,95,96,97,98,99]
# bs_list = [100,200,300,400,500,600,700,800,900,1000]
# n
# bs_list = [100,200,300,400,500,600,700,800]
bs_list = [100,200,300,400]
for bs in tqdm(bs_list):
    print(f'bs{bs}', end=' ')
    for p in p_list:
        print(f'p{p}', end=' ')
        
        layer_knb_dict_max_dict, layer_knb_dict_mean_dict = {}, {}
        for weight_name, gas_array in knb_gas.items():
            print(weight_name, end=' ')
            gas_scores = gas_array[0:bs]
            # max
            gas_max = np.max(gas_scores, axis=0)
            # w [14096, 4096]
            # (885, 28, 4096) -> [28,4096]
            knb_idx_max = np.argwhere((gas_max > np.percentile(gas_max, p)) & (gas_max > 0))
            layer_knb_dict_max = {}
            for l in range(n_layers):
                layer_knb_dict_max[l] = knb_idx_max[knb_idx_max[:, 0] == l][:, 1].tolist()
            layer_knb_dict_max_dict[weight_name] = layer_knb_dict_max
            # mean
            gas_mean = np.mean(gas_scores, axis=0)
            knb_idx_mean = np.argwhere((gas_mean > np.percentile(gas_mean, p)) & (gas_mean > 0))
            layer_knb_dict_mean = {}
            for l in range(n_layers):
                layer_knb_dict_mean[l] = knb_idx_mean[knb_idx_mean[:, 0] == l][:, 1].tolist()
            layer_knb_dict_mean_dict[weight_name] = layer_knb_dict_mean
        
        # save knb_dict
        with open(f'{full_path}/n{bs}-p{p}-max.json', 'w') as f:
            json.dump(layer_knb_dict_max_dict, f, indent=4)
        with open(f'{full_path}/n{bs}-p{p}-mean.json', 'w') as f:
            json.dump(layer_knb_dict_mean_dict, f, indent=4)
        print()

  0%|          | 0/4 [00:00<?, ?it/s]

bs100 p90 mlp.up_proj mlp.down_proj 
p91 mlp.up_proj mlp.down_proj 
p92 mlp.up_proj mlp.down_proj 
p93 mlp.up_proj mlp.down_proj 
p94 mlp.up_proj mlp.down_proj 
p95 mlp.up_proj mlp.down_proj 
p96 mlp.up_proj mlp.down_proj 
p97 mlp.up_proj mlp.down_proj 
p98 mlp.up_proj mlp.down_proj 
p99 mlp.up_proj mlp.down_proj 

 25%|██▌       | 1/4 [00:02<00:06,  2.02s/it]


bs200 p90 mlp.up_proj mlp.down_proj 
p91 mlp.up_proj mlp.down_proj 
p92 mlp.up_proj mlp.down_proj 
p93 mlp.up_proj mlp.down_proj 
p94 mlp.up_proj mlp.down_proj 
p95 mlp.up_proj mlp.down_proj 
p96 mlp.up_proj mlp.down_proj 
p97 mlp.up_proj mlp.down_proj 
p98 mlp.up_proj mlp.down_proj 
p99 mlp.up_proj 

 50%|█████     | 2/4 [00:05<00:05,  2.75s/it]

mlp.down_proj 
bs300 p90 mlp.up_proj mlp.down_proj 
p91 mlp.up_proj mlp.down_proj 
p92 mlp.up_proj mlp.down_proj 
p93 mlp.up_proj mlp.down_proj 
p94 mlp.up_proj mlp.down_proj 
p95 mlp.up_proj mlp.down_proj 
p96 mlp.up_proj mlp.down_proj 
p97 mlp.up_proj mlp.down_proj 
p98 mlp.up_proj mlp.down_proj 
p99 mlp.up_proj 

 75%|███████▌  | 3/4 [00:09<00:03,  3.55s/it]

mlp.down_proj 
bs400 p90 mlp.up_proj mlp.down_proj 
p91 mlp.up_proj mlp.down_proj 
p92 mlp.up_proj mlp.down_proj 
p93 mlp.up_proj mlp.down_proj 
p94 mlp.up_proj mlp.down_proj 
p95 mlp.up_proj mlp.down_proj 
p96 mlp.up_proj mlp.down_proj 
p97 mlp.up_proj mlp.down_proj 
p98 mlp.up_proj mlp.down_proj 
p99 mlp.up_proj 

100%|██████████| 4/4 [00:15<00:00,  3.89s/it]

mlp.down_proj 


# all dataset
bs = n
for p in p_list:
    print(f'p{p}', end=' ')
    layer_knb_dict_max_dict = {}
    for weight_name, gas_array in knb_gas.items():
        print(weight_name, end=' ')
        gas_scores = gas_array
        # max
        gas_max = np.max(gas_scores, axis=0)
        knb_idx_max = np.argwhere((gas_max > np.percentile(gas_max, p)) & (gas_max > 0))
        layer_knb_dict_max = {}
        for l in range(n_layers):
            layer_knb_dict_max[l] = knb_idx_max[knb_idx_max[:, 0] == l][:, 1].tolist()
        layer_knb_dict_max_dict[weight_name] = layer_knb_dict_max
    
    # save knb_dict
    with open(f'{full_path}/n{bs}-p{p}-max.json', 'w') as f:
        json.dump(layer_knb_dict_max_dict, f, indent=4)
    print()

n = gas_array.shape[0]
n_layers = gas_array.shape[1]
# 分位数,选取gas分数最高的1-p%作为knb
p_list = [90,91,92,93,94,95,96,97,98,99]
# p_list2 = [99.1,99.2,99.3,99.4,99.5,99.6,99.7,99.8,99.9]
# p_list = p_list1 + p_list2
# bs_list = [1,2,4,6,8,10,20,30,40,50,60,70,80,90,100]
bs_list = [1]
for bs in tqdm(bs_list):
    print(f'bs{bs}', end=' ')
    for p in p_list:
        print(f'p{p}', end=' ')
        bs_weight_layer_knb_dict_max_list = []
        for i in range(0, n, bs):
            j = min(i+bs, n)
            layer_knb_dict_max_dict = {}
            for weight_name, gas_array in knb_gas.items():
                print(weight_name, end=' ')
                gas_scores = gas_array[i:j]
                # max
                gas_max = np.max(gas_scores, axis=0)
                knb_idx_max = np.argwhere((gas_max > np.percentile(gas_max, p)) & (gas_max > 0))
                layer_knb_dict_max = {}
                for l in range(n_layers):
                    layer_knb_dict_max[l] = knb_idx_max[knb_idx_max[:, 0] == l][:, 1].tolist()
                layer_knb_dict_max_dict[weight_name] = layer_knb_dict_max
            bs_weight_layer_knb_dict_max_list.append(layer_knb_dict_max_dict)
        # save knb_dict
        with open(f'{full_path}/bs{bs}-p{p}-max.json', 'w') as f:
            json.dump(bs_weight_layer_knb_dict_max_list, f, indent=4)
        print()